In [34]:
import numpy as np
import json_tricks
answer = {}

inputs = json_tricks.load('inputs.json')


# Matrix Product II

$$A_{5 \times 10} \  B_{10 \times 2}\ C_{2 \times 30}\ D_{30 \times 3}\ E_{3 \times 9}$$

1. What will be the shape of the resulting matrix?
2. How many multiplication of numbers are required at best?

In [35]:
answer['task1'] = {
    '1': (5, 9),
    '2': 424
}



# Numpy expression

Using Numpy, write a function that calculates the 
following expression:

$$\exp(A^T(B + 2C) + 3I) \mathbf x,$$

where $I$ is an identity matrix of the necessary shape.

In [ ]:

def numpy_expression(A, B, C, x):
    N = A.shape[-1]
    M = B.shape[-1]
    mat = (A.T @ (B + (2 * C))) + (3 * np.eye(N, M))
    
    res = np.exp(mat) @ x

    return res.astype(np.float64)

In [37]:
answer['task2'] = []
for one_input in inputs['task2']:
    answer['task2'].append(numpy_expression(**one_input))


/tmp/ipykernel_5865/1869993749.py:8: RuntimeWarning: overflow encountered in cast
  return res.astype(np.float32)


# Einstein's Rule

In *Tensor Algebra*, a direct generalization of the Linear Algebra to the case of $N$-dimentional tables called *tensors* (normal matrix), the Einstein's rule exists.

It works as follows: if you see a duplicating upper and lower index in the formula, that means, this index convolves.

For example, the following tensor expression, summation and matrix product are equivalent:

$$a_k^l b_l^m = \sum_{l=1}^L a_k^l b_l^m = AB$$

In this notation subscript means row index and superscript means column index.

<details>
<summary> Note </summary>

> [!NOTE]
> Also at some point it will be important to know that:
> * lower index represents a contravariant dimension of a
> tensor
> * upper index represents a covariant dimension 
> of tensor. But let us omit this part for now.

</details>

# Task

Calculate the following expression written using Einstein's 
rule:

$$a_k^m b_m^n c_n^o d_l^k$$

In [38]:
def einsteins_rule(A, B, C, D):
    res = (D @ (A @ B @ C)) 
    return res

In [39]:
answer['task3'] = []
for one_input in inputs['task3']:
    answer['task3'].append(einsteins_rule(**one_input))

# answer['task3']

# Diagonal Matrix Product

You are given two square matrices: $A$ and $D$, where $A$ is a 
full matrix and $D$ is a diagonal matrix:

$$
A = \begin{bmatrix}
- & \mathbf a_1 & - \\
& \vdots & \\
- & \mathbf a_N & - \\
\end{bmatrix}
$$

$$
D = \textrm{diag}(d_1, d_2, \dots, d_N) = \begin{bmatrix}
d_1 & & & & \\
& d_2 & & & \\
& & d_3 & & \\
& & & \ddots & \\
& & & & d_N 
\end{bmatrix}
$$

Write a program to calculate the result of $DA$ and $AD$ in 
the fastest possible way.

In [40]:
def diag_prod_DA(A, D):
    res = A.copy()
    D_diag = np.diag(D)

    for i in range(A.shape[0]):
        res[i, :] = D_diag[i] * res[i, :]

    return res

def diag_prod_AD(A, D):
    res = A.copy()

    D_diag = np.diag(D)
    for i in range(A.shape[1]):
        res[:, i] = D_diag[i] * res[:, i]
    
    return res

In [41]:
answer['task4_1'] = []
answer['task4_2'] = []
for one_input in inputs['task4']:
    answer['task4_1'].append(diag_prod_DA(**one_input))
    answer['task4_2'].append(diag_prod_AD(**one_input))

# Sparse Matrix Product

You are given two matrices of the same shape: $A$ and $B$. Matrix $A$ is full
and is given in the form of `numpy.ndarray`.

The second matrix $B$ is **sparse**. That means that the 
majority of the items are equal to $0$ except for $M$. This matrix is given
as a set of non-zero elements of this matrix in form of $3 \times M$ `numpy.ndarray` as row-column-value tuple (COO sparse matrix form):

$$
\begin{bmatrix}
r_1 & c_1 & v_1 \\
r_2 & c_2 & v_2 \\
& \vdots & \\
r_M & c_M & v_M \\
\end{bmatrix}
$$

If in this struct two items correspond to the same location, consider the latter is correct.

Write the most efficient program that calculates $AB$.

Also return the ratio between the number of multiplication operations that are needed to calculate the sparse product and the number of operations for full product.

In [42]:
def sparse_matrix_product(A, B_sparse):
    ops = 0

    N, M = A.shape[0], int(B_sparse[1].max()) + 1
    res = np.zeros((N, M))

    not_duplicated = {}
    for r, c, v in B_sparse.T:
        not_duplicated[int(r), int(c)] = v

    for (r, c), v in not_duplicated.items():
        r, c = int(r), int(c)
        ops += 1
        res[:, c] += A[:, r] * v 

    ratio = (ops * N) / (A.shape[0] * A.shape[1] * M)
    print(res)
    return res, ratio


In [43]:
answer['task5'] = []
for one_input in inputs['task5']:
    answer['task5'].append(sparse_matrix_product(**one_input))

[[  6. -60.  10.  63.  66.]
 [ -1. -98.  -7. -14. -39.]
 [ 34. 140.   6.  14. -30.]
 [-48. -50.  40. -49.  -8.]
 [ 26.   0. -18. -14. -70.]
 [-36. -34. -60. -35.  80.]
 [-25. -32.  -7.  14.  77.]
 [-72. -22. -24. -63.  80.]]
[[ -18.  116.    0.    3.]
 [ -22. -126.    0.  -32.]]
[[  0.  24.   0.  -2.]
 [  0.  27.   0.   4.]
 [  0. -18.   0.  12.]
 [  0.  -3.   0.  16.]]
[[   0.   -9.    0.    0.  -18.  -54.  -34.]
 [  28.   24.    0.    0.   26.  -20.   62.]
 [ -12.   49.    0.    0.  -24.    0.   50.]
 [ -32.   18.    0.    0.   88.  -40.   38.]
 [  34.    9.    0.    0.  -76.  -53.   34.]
 [ -14.  -27.    0.    0.   50.   37. -108.]]
[[ 101.  -54.   97.  100.   22.  -36.  -87. -160.]
 [ -12.   98.  -78. -119.    6.   32.   36.   51.]
 [-106.  -56.  -10.  -26.   19.  -92.  -39.   21.]]
[[ -59.    0.    9.  -22.  -21.  103.   77.]
 [ -44.   14.   93.  -26.   22. -100.   57.]
 [  32.  -17.  -41.   24.   25.    1.  -61.]]
[[ -71.   58.   42.   49.   36.  -22.]
 [ -39.   60.  -44.   -1.  

In [44]:
json_tricks.dump(answer, '.answer.json')

ValueError: Out of range float values are not JSON compliant